In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [202]:
mangas = pd.read_csv('User lists/sorted_manga_list.csv')
users = pd.read_csv('User lists/all_user_lists_v2.csv')

In [203]:
mangas_arr = np.array()

,manga_id,title,score,start_date,Action,Adventure,Award Winning,Boys Love,Comedy,Drama,...,Time Travel,Vampire,Video Game,Visual Arts,Workplace,Josei,Kids,Seinen,Shoujo,Shounen
0,2,Berserk,9.47,1989,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,13,One Piece,9.22,1997,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,9.30,2004,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,4632,Oyasumi Punpun,9.02,2007,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,25,Fullmetal Alchemist,9.03,2001,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,54387,Saiteihen no Otoko,6.14,2011,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1206,110727,Gigant,5.92,2017,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1207,45613,Pupa,5.89,2011,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1208,117133,Samurai 8: Hachimaru Den,5.75,2019,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
lens = mangas.merge(users, on='title')

genrecols = mangas.iloc[:,4:].columns.values.tolist()

# RESHAPE DF BY MELTING (WIDE TO LONG), SELECTING ONLY NEEDED FIELDS    
mdf = pd.melt(lens[['user', 'score_y'] + genrecols], 
              id_vars=['user', 'score_y'], var_name='genre')

# # FILTER FOR VALUE = 1 AND THREE NEEDED COLUMNS
mdf = mdf[mdf['value']==1][['user', 'score_y', 'genre']]

# # RUN PIVOTED AGGREGATION    
user_avg = pd.pivot_table(mdf, columns = ['genre'], index = ['user'], 
                    values = ['score_y'], aggfunc = 'mean')

user_avg.fillna(0, inplace=True)
user_avg

score_y                                                     \
genre           Action Adult Cast Adventure Anthropomorphic Award Winning   
user                                                                        
--EYEPATCH--  8.846154   9.666667  9.555556             9.0      9.153846   
--Loki--      5.666667   0.000000  0.000000             0.0      4.500000   
--Magma--     6.777778   0.000000  6.000000             0.0      3.333333   
--Rocko--     0.000000   0.000000  0.000000             0.0      0.000000   
--William--   6.033333   7.750000  7.111111             0.0      5.875000   
...                ...        ...       ...             ...           ...   
zzz_roza      3.538462   0.000000  4.000000             0.0      4.777778   
zzzack        8.500000   0.000000  0.000000             0.0      0.000000   
zzzerosleep   9.600000   0.000000  0.000000             0.0     10.000000   
zzzmohamed    5.461538   0.000000  5.642857             0.0      5.700000   
zzzzkung      6.333333   0.000000  7.000000             0.0      8.000000   

                                                                ...  \
genre        Boys Love CGDCT Childcare Combat Sports    Comedy  ...   
user                                                            ...   
--EYEPATCH--       0.0   0.0       8.0           9.0  9.000000  ...   
--Loki--           0.0   0.0       0.0           0.0  8.000000  ...   
--Magma--          0.0   0.0       0.0           0.0  6.750000  ...   
--Rocko--          0.0   0.0       0.0           0.0  0.000000  ...   
--William--        0.0   0.0       4.5           0.0  2.125000  ...   
...                ...   ...       ...           ...       ...  ...   
zzz_roza           6.0   0.0       8.0           0.0  5.666667  ...   
zzzack             0.0   0.0       0.0           0.0  0.000000  ...   
zzzerosleep        0.0   0.0       0.0           0.0  0.000000  ...   
zzzmohamed         0.0   0.0       0.0           0.0  3.818182  ...   
zzzzkung           0.0   0.0       0.0           0.0  6.400000  ...   

                                                                       \
genre        Super Power Supernatural   Survival Suspense Team Sports   
user                                                                    
--EYEPATCH--         9.0     8.400000   7.000000     6.00         9.0   
--Loki--             0.0     0.000000   0.000000     8.00         0.0   
--Magma--            8.0     7.250000  10.000000     9.00         0.0   
--Rocko--            0.0     0.000000   0.000000     0.00         0.0   
--William--          6.0     5.312500   3.000000     0.00         0.0   
...                  ...          ...        ...      ...         ...   
zzz_roza             0.0     4.750000   0.000000    10.00         0.0   
zzzack               0.0     8.000000   0.000000     0.00        10.0   
zzzerosleep          0.0     9.500000   0.000000     0.00        10.0   
zzzmohamed           7.5     4.083333   5.666667     5.75         0.0   
zzzzkung             0.0     5.333333   0.000000     0.00         8.5   

                                                                   
genre        Time Travel Vampire Video Game Visual Arts Workplace  
user                                                               
--EYEPATCH--         0.0     0.0        0.0         0.0      10.0  
--Loki--             0.0     0.0        0.0         0.0       0.0  
--Magma--            0.0    10.0        0.0         0.0       0.0  
--Rocko--            0.0     0.0        0.0         0.0       0.0  
--William--          0.0     5.5        0.0         0.0       0.0  
...                  ...     ...        ...         ...       ...  
zzz_roza             0.0     0.0        0.0         8.0       0.0  
zzzack               0.0     0.0        0.0         0.0       0.0  
zzzerosleep          0.0     0.0        0.0         0.0       0.0  
zzzmohamed           8.0     0.0        0.0         0.0       0.0  
zzzzkung             0.0     0.0  

In [4]:
x2 = lens.drop(['manga_id','title','score_y','user','id','status'], axis=1)
x2 

,score_x,start_date,Action,Adventure,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,...,Time Travel,Vampire,Video Game,Visual Arts,Workplace,Josei,Kids,Seinen,Shoujo,Shounen
0,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874612,5.75,2019,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
874613,5.75,2019,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
874614,5.75,2019,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
874615,5.75,2019,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
x = pd.DataFrame(user_avg.values, columns=user_avg.columns.get_level_values(1))
x = pd.concat([x,pd.Series(user_avg.index)], axis=1)
x

,Action,Adult Cast,Adventure,Anthropomorphic,Award Winning,Boys Love,CGDCT,Childcare,Combat Sports,Comedy,...,Supernatural,Survival,Suspense,Team Sports,Time Travel,Vampire,Video Game,Visual Arts,Workplace,user
0,8.846154,9.666667,9.555556,9.0,9.153846,0.0,0.0,8.0,9.0,9.000000,...,8.400000,7.000000,6.00,9.0,0.0,0.0,0.0,0.0,10.0,--EYEPATCH--
1,5.666667,0.000000,0.000000,0.0,4.500000,0.0,0.0,0.0,0.0,8.000000,...,0.000000,0.000000,8.00,0.0,0.0,0.0,0.0,0.0,0.0,--Loki--
2,6.777778,0.000000,6.000000,0.0,3.333333,0.0,0.0,0.0,0.0,6.750000,...,7.250000,10.000000,9.00,0.0,0.0,10.0,0.0,0.0,0.0,--Magma--
3,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,--Rocko--
4,6.033333,7.750000,7.111111,0.0,5.875000,0.0,0.0,4.5,0.0,2.125000,...,5.312500,3.000000,0.00,0.0,0.0,5.5,0.0,0.0,0.0,--William--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33813,3.538462,0.000000,4.000000,0.0,4.777778,6.0,0.0,8.0,0.0,5.666667,...,4.750000,0.000000,10.00,0.0,0.0,0.0,0.0,8.0,0.0,zzz_roza
33814,8.500000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,8.000000,0.000000,0.00,10.0,0.0,0.0,0.0,0.0,0.0,zzzack
33815,9.600000,0.000000,0.000000,0.0,10.000000,0.0,0.0,0.0,0.0,0.000000,...,9.500000,0.000000,0.00,10.0,0.0,0.0,0.0,0.0,0.0,zzzerosleep
33816,5.461538,0.000000,5.642857,0.0,5.700000,0.0,0.0,0.0,0.0,3.818182,...,4.083333,5.666667,5.75,0.0,8.0,0.0,0.0,0.0,0.0,zzzmohamed


In [6]:
x1 = users.merge(x, on='user')
x1.drop(['user','id','title','status','score'],axis=1, inplace=True)
x1

,Action,Adult Cast,Adventure,Anthropomorphic,Award Winning,Boys Love,CGDCT,Childcare,Combat Sports,Comedy,...,Super Power,Supernatural,Survival,Suspense,Team Sports,Time Travel,Vampire,Video Game,Visual Arts,Workplace
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.916667,0.0,6.0,0.0,5.666667,0.0,0.0,0.0,0.0,6.4,...,8.0,6.333333,7.666667,9.0,9.0,0.0,0.0,8.0,0.0,0.0
3,6.916667,0.0,6.0,0.0,5.666667,0.0,0.0,0.0,0.0,6.4,...,8.0,6.333333,7.666667,9.0,9.0,0.0,0.0,8.0,0.0,0.0
4,6.916667,0.0,6.0,0.0,5.666667,0.0,0.0,0.0,0.0,6.4,...,8.0,6.333333,7.666667,9.0,9.0,0.0,0.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874612,2.500000,0.0,5.0,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,10.000000,0.000000,5.0,0.0,0.0,0.0,0.0,0.0,0.0
874613,2.500000,0.0,5.0,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,10.000000,0.000000,5.0,0.0,0.0,0.0,0.0,0.0,0.0
874614,2.500000,0.0,5.0,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,10.000000,0.000000,5.0,0.0,0.0,0.0,0.0,0.0,0.0
874615,2.500000,0.0,5.0,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,10.000000,0.000000,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
yy = np.array(pd.melt(users[['user','score','title']], id_vars=['user','score'], var_name='title')['score'])
yy

array([0, 0, 8, ..., 0, 0, 0], dtype=int64)

In [8]:
matrix = users[['user','title','score']].pivot_table(index='user', columns='title', values='score')
matrix

title,"""Kare"" First Love","""Oshi no Ko""",+Anima,.hack//Tasogare no Udewa Densetsu,07-Ghost,1-nen A-gumi no Monster,14-sai no Koi,16 Life,17-21: Fujimoto Tatsuki Tanpenshuu,20th Century Boys,...,Zetsuen no Tempest,Zettai Heiwa Daisakusen,Zettai Kareshi.,Zom 100: Zombie ni Naru made ni Shitai 100 no Koto,Zombie Powder.,Zombie-Loan,ib: Instant Bullet,xxxHOLiC,Übel Blatt,ənígmə
user,,,,,,,,,,,,,,,,,,,,,
--EYEPATCH--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Loki--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Magma--,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Rocko--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--William--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzz_roza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzerosleep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
mangas = mangas[mangas['title'].isin(matrix.T.index)]

In [10]:
Y = np.array(matrix)
Y

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, 10., nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  0., nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [11]:
item_train_save = x2
user_train_save = x1
#mangas = mangas.drop(['title','manga_id'], axis=1)

scalerItem = StandardScaler()
scalerItem.fit(x2)
x2 = scalerItem.transform(x2)

scalerUser = StandardScaler()
scalerUser.fit(x1)
x1 = scalerUser.transform(x1)

print(np.allclose(item_train_save, scalerItem.inverse_transform(x2)))
print(np.allclose(user_train_save, scalerUser.inverse_transform(x1)))

True
True


In [12]:
x2_train, x2_test = train_test_split(x2, train_size=0.8, shuffle=True, random_state=1)
x1_train, x1_test = train_test_split(x1, train_size=0.80, shuffle=True, random_state=1)
yy_train, yy_test   = train_test_split(yy, train_size=0.80, shuffle=True, random_state=1)

In [13]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(yy_train.reshape(-1, 1))
ynorm_train = scaler.transform(yy_train.reshape(-1, 1))
ynorm_test = scaler.transform(yy_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(699693, 1) (174924, 1)


In [14]:
num_user_features =  user_train_save.shape[1]
num_item_features = item_train_save.shape[1]

In [15]:
num_user_features, num_item_features

(69, 71)

In [124]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)),
    tf.keras.layers.Dense(num_outputs) 
], name = 'user_layers')

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)),
    tf.keras.layers.Dense(num_outputs)
], name='item_layers')

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features), name='Input User')
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features), name='Input Item')
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1, name='dot_produce')([vu, vm])

# specify the inputs and output of the model
model = keras.Model([input_user, input_item], output)


model.summary() 

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input User (InputLayer)     [(None, 69)]                 0         []                            
                                                                                                  
 Input Item (InputLayer)     [(None, 71)]                 0         []                            
                                                                                                  
 user_layers (Sequential)    (None, 32)                   3296      ['Input User[0][0]']          
                                                                                                  
 item_layers (Sequential)    (None, 32)                   3360      ['Input Item[0][0]']          
                                                                                           

In [125]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=opt,
              loss=cost_fn)

In [126]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)

In [127]:
tf.random.set_seed(1)
model.fit([x1_train, x2_train], ynorm_train, epochs=10, callbacks=[early_stopping])

Epoch 1/10
21866/21866 [==============================] - 18s 809us/step - loss: 0.4926
Epoch 2/10
21866/21866 [==============================] - 18s 811us/step - loss: 0.4774
Epoch 3/10
21866/21866 [==============================] - 17s 799us/step - loss: 0.4776
Epoch 4/10
21866/21866 [==============================] - 17s 797us/step - loss: 0.4770
Epoch 5/10
21866/21866 [==============================] - 17s 794us/step - loss: 0.4765
Epoch 6/10
21866/21866 [==============================] - 18s 803us/step - loss: 0.4767
Epoch 7/10
21866/21866 [==============================] - 17s 796us/step - loss: 0.4767


In [128]:
model.evaluate([x1_test, x2_test], yy_test)

5467/5467 [==============================] - 4s 629us/step - loss: 38.7511


38.7510986328125

In [129]:
mylist = pd.read_csv('User lists/mylist.csv')
mylist

,user,id,title,status,score
0,Haelios,1224,3-gatsu no Lion,0,9
1,Haelios,103851,5-toubun no Hanayome,0,8
2,Haelios,32,666 Satan,0,8
3,Haelios,112115,Act-age,0,9
4,Haelios,25132,Akame ga Kill!,0,8
...,...,...,...,...,...
144,Haelios,52,Whistle!,0,10
145,Haelios,653,Yakitate!! Japan,0,7
146,Haelios,100128,Yakusoku no Neverland,0,7
147,Haelios,121480,Yofukashi no Uta,1,6


In [134]:
n_users = pd.concat([users,mylist])

In [135]:
mylens = mangas.merge(n_users, on='title')

genrecols = mangas.iloc[:,4:].columns.values.tolist()

# RESHAPE DF BY MELTING (WIDE TO LONG), SELECTING ONLY NEEDED FIELDS    
mdf = pd.melt(mylens[['user', 'score_y'] + genrecols], 
              id_vars=['user', 'score_y'], var_name='genre')

# # FILTER FOR VALUE = 1 AND THREE NEEDED COLUMNS
mdf = mdf[mdf['value']==1][['user', 'score_y', 'genre']]

# # RUN PIVOTED AGGREGATION    
user_avg = pd.pivot_table(mdf, columns = ['genre'], index = ['user'], 
                    values = ['score_y'], aggfunc = 'mean')

user_avg.fillna(0, inplace=True)
user_avg

score_y                                                     \
genre           Action Adult Cast Adventure Anthropomorphic Award Winning   
user                                                                        
--EYEPATCH--  8.846154   9.666667  9.555556             9.0      9.153846   
--Loki--      5.666667   0.000000  0.000000             0.0      4.500000   
--Magma--     6.777778   0.000000  6.000000             0.0      3.333333   
--Rocko--     0.000000   0.000000  0.000000             0.0      0.000000   
--William--   6.033333   7.750000  7.111111             0.0      5.875000   
...                ...        ...       ...             ...           ...   
zzz_roza      3.538462   0.000000  4.000000             0.0      4.777778   
zzzack        8.500000   0.000000  0.000000             0.0      0.000000   
zzzerosleep   9.600000   0.000000  0.000000             0.0     10.000000   
zzzmohamed    5.461538   0.000000  5.642857             0.0      5.700000   
zzzzkung      6.333333   0.000000  7.000000             0.0      8.000000   

                                                                ...  \
genre        Boys Love CGDCT Childcare Combat Sports    Comedy  ...   
user                                                            ...   
--EYEPATCH--       0.0   0.0       8.0           9.0  9.000000  ...   
--Loki--           0.0   0.0       0.0           0.0  8.000000  ...   
--Magma--          0.0   0.0       0.0           0.0  6.750000  ...   
--Rocko--          0.0   0.0       0.0           0.0  0.000000  ...   
--William--        0.0   0.0       4.5           0.0  2.125000  ...   
...                ...   ...       ...           ...       ...  ...   
zzz_roza           6.0   0.0       8.0           0.0  5.666667  ...   
zzzack             0.0   0.0       0.0           0.0  0.000000  ...   
zzzerosleep        0.0   0.0       0.0           0.0  0.000000  ...   
zzzmohamed         0.0   0.0       0.0           0.0  3.818182  ...   
zzzzkung           0.0   0.0       0.0           0.0  6.400000  ...   

                                                                       \
genre        Super Power Supernatural   Survival Suspense Team Sports   
user                                                                    
--EYEPATCH--         9.0     8.400000   7.000000     6.00         9.0   
--Loki--             0.0     0.000000   0.000000     8.00         0.0   
--Magma--            8.0     7.250000  10.000000     9.00         0.0   
--Rocko--            0.0     0.000000   0.000000     0.00         0.0   
--William--          6.0     5.312500   3.000000     0.00         0.0   
...                  ...          ...        ...      ...         ...   
zzz_roza             0.0     4.750000   0.000000    10.00         0.0   
zzzack               0.0     8.000000   0.000000     0.00        10.0   
zzzerosleep          0.0     9.500000   0.000000     0.00        10.0   
zzzmohamed           7.5     4.083333   5.666667     5.75         0.0   
zzzzkung             0.0     5.333333   0.000000     0.00         8.5   

                                                                   
genre        Time Travel Vampire Video Game Visual Arts Workplace  
user                                                               
--EYEPATCH--         0.0     0.0        0.0         0.0      10.0  
--Loki--             0.0     0.0        0.0         0.0       0.0  
--Magma--            0.0    10.0        0.0         0.0       0.0  
--Rocko--            0.0     0.0        0.0         0.0       0.0  
--William--          0.0     5.5        0.0         0.0       0.0  
...                  ...     ...        ...         ...       ...  
zzz_roza             0.0     0.0        0.0         8.0       0.0  
zzzack               0.0     0.0        0.0         0.0       0.0  
zzzerosleep          0.0     0.0        0.0         0.0       0.0  
zzzmohamed           8.0     0.0        0.0         0.0       0.0  
zzzzkung             0.0     0.0  

In [145]:
my_vect = user_avg[user_avg.index=='Haelios']
my_vect.shape

(1, 69)

In [147]:
my_vect = np.tile(my_vect,(1067,1))
print(my_vect.shape)
suser_vecs = scalerUser.transform(my_vect)

(1067, 69)


D:\Documents\projperso\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [148]:
suser_vecs.shape

(1067, 69)

In [206]:
xy = lens.drop(['score_y','user','id','status'], axis=1)
xy = xy.drop_duplicates(subset=np.array(xy.columns),keep='first')
xy

,manga_id,title,score_x,start_date,Action,Adventure,Award Winning,Boys Love,Comedy,Drama,...,Time Travel,Vampire,Video Game,Visual Arts,Workplace,Josei,Kids,Seinen,Shoujo,Shounen
0,2,Berserk,9.47,1989,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
12327,13,One Piece,9.22,1997,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24721,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,9.30,2004,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
31631,4632,Oyasumi Punpun,9.02,2007,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
39833,25,Fullmetal Alchemist,9.03,2001,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873054,28697,Big Order,6.23,2011,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
873208,54387,Saiteihen no Otoko,6.14,2011,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
873379,110727,Gigant,5.92,2017,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
873990,45613,Pupa,5.89,2011,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [207]:
manga_arr = np.array(xy['title'])
manga_arr

array(['Berserk', 'One Piece',
       'JoJo no Kimyou na Bouken Part 7: Steel Ball Run', ..., 'Gigant',
       'Pupa', 'Samurai 8: Hachimaru Den'], dtype=object)

In [190]:
xx = lens.drop(['title','score_y','user','id','status'], axis=1)
xx = xx.drop_duplicates(subset=np.array(xx.columns),keep='first')
xx.drop(['manga_id'],inplace=True,axis=1)

In [191]:
sitem_vecs = scalerItem.transform(xx)

In [195]:
xx

,score_x,start_date,Action,Adventure,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,...,Time Travel,Vampire,Video Game,Visual Arts,Workplace,Josei,Kids,Seinen,Shoujo,Shounen
0,9.47,1989,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
12327,9.22,1997,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24721,9.30,2004,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
31631,9.02,2007,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
39833,9.03,2001,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873054,6.23,2011,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
873208,6.14,2011,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
873379,5.92,2017,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
873990,5.89,2011,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
# make a prediction
y_p = model.predict([suser_vecs, sitem_vecs])

# unscale y prediction 
y_pu = scaler.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = manga_arr[sorted_index]  #using unscaled vectors for display

34/34 [==============================] - 0s 606us/step


In [216]:
sorted_items[:5]

array(['Oresama Teacher', 'Ao no Flag',
       '.hack//Tasogare no Udewa Densetsu', 'B-gata H-kei',
       'Zelda no Densetsu: Majora no Kamen'], dtype=object)